In [1]:
import numpy as np
import pandas as pd

# Settings

In [2]:
settings = { 
            'data_size': int(2e5), # size of data subset after shuffle is performed
            'test_size': 0.25, # fraction of data set to be assigned as test data
            'save_env': False, # save environment
            'del': True # Delete variables that are no longer needed to proceed in computations to save place
           }

# Load Data: load dataset, shuffle it and take subset

In [3]:
PATH = "../data/"

In [4]:
data_full = pd.read_csv(f'{PATH}train.tsv', sep='\t')

In [5]:
data_full.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


In [6]:
len(data_full)

1482535

In [7]:
from sklearn.utils import shuffle
data_shuffled = shuffle(data_full)

In [8]:
if (settings['del']):
    del data_full

In [9]:
data = data_shuffled.iloc[:settings['data_size'], :]

In [10]:
if (settings['del']):
    del data_shuffled

In [11]:
len(data)

200000

In [12]:
data.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
1237964,1237964,"Light Up ""M""",2,Home/Home Décor/Home Décor Accents,NaN,14.0,0,Mint condition. Super cute decor
623176,623176,ZARA : graphic tee,2,Women/Tops & Blouses/T-Shirts,ZARA,17.0,0,"""Self confidence is the best outfit"" tee"
203562,203562,Old Navy Active,3,Women/Tops & Blouses/T-Shirts,Old Navy,5.0,1,See through long sleeve work out shirt.
538344,538344,IKEA Sanela Curtains,2,Home/Home Décor/Window Treatments,IKEA,24.0,0,One set of beautiful dark brown velvet panel c...
179589,179589,Old Navy Swimsuit,2,Women/Swimwear/Two-Piece,Old Navy,8.0,1,"New, never worn only tried on. Reddish pink co..."


# Data preparation

## Item Category

Split category_name into main_cat, subcat_1 and subcat_2

In [13]:
# reference: BuryBuryZymon at https://www.kaggle.com/maheshdadhich/i-will-sell-everything-for-free-0-55
def split_cat(text):
    try: return text.split("/")
    except: return ("No Label", "No Label", "No Label")

In [14]:
data.loc[:,'main_cat'], data.loc[:,'subcat_1'], data.loc[:,'subcat_2'] = \
zip(*data.loc[:,'category_name'].apply(lambda x: split_cat(x)))
# data.head()

In [15]:
data.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,main_cat,subcat_1,subcat_2
1237964,1237964,"Light Up ""M""",2,Home/Home Décor/Home Décor Accents,NaN,14.0,0,Mint condition. Super cute decor,Home,Home Décor,Home Décor Accents
623176,623176,ZARA : graphic tee,2,Women/Tops & Blouses/T-Shirts,ZARA,17.0,0,"""Self confidence is the best outfit"" tee",Women,Tops & Blouses,T-Shirts
203562,203562,Old Navy Active,3,Women/Tops & Blouses/T-Shirts,Old Navy,5.0,1,See through long sleeve work out shirt.,Women,Tops & Blouses,T-Shirts
538344,538344,IKEA Sanela Curtains,2,Home/Home Décor/Window Treatments,IKEA,24.0,0,One set of beautiful dark brown velvet panel c...,Home,Home Décor,Window Treatments
179589,179589,Old Navy Swimsuit,2,Women/Swimwear/Two-Piece,Old Navy,8.0,1,"New, never worn only tried on. Reddish pink co...",Women,Swimwear,Two-Piece


## Overview

In [16]:
print("Number of unique fields:\n")

print("main_cat: \t%d" % data['main_cat'].nunique())
print("subcat_1: \t%d" % data['subcat_1'].nunique())
print("subcat_2: \t%d" % data['subcat_2'].nunique())
print("brand_name: \t%d" % data['brand_name'].nunique())
print()

print("%d items have no category" % len(data.loc[data['main_cat'] == 'No Label']))
print("%d items have no brand" % data['brand_name'].isna().sum())

Number of unique fields:

main_cat: 	11
subcat_1: 	114
subcat_2: 	748
brand_name: 	2602

859 items have no category
85425 items have no brand


## Numerically represent features

### train_id: copy

Create new DataFrame called data_num for numerical representations

In [17]:
data_num = pd.DataFrame(data.loc[:,'train_id'], columns=['train_id'])

### name: represent name by its length as name_len

In [18]:
data_num.loc[:,'name_len'] = data['name'].str.len()

### item_condition_id, price, shipping: copy

In [19]:
data_num[['item_condition_id', 'price', 'shipping']] = data.loc[:,['item_condition_id', 'price', 'shipping']]

### item_description

Represent item_description by its length as item_description_len

In [20]:
item_description_len = data.loc[:,'item_description'].str.len()
data_num['item_description_len'] = item_description_len

# Replace NaN in data_num.item_description_len column by zeros
data_num['item_description_len'] = data_num['item_description_len'].fillna(0)

# Change data type of this column to uint16 provided the max val is less than 65535
if (data_num.item_description_len.max() < 65535):
    data_num['item_description_len'] = data_num['item_description_len'].astype(np.uint16)

In [21]:
# data_num.head()

In [22]:
# data.head()

### Define function for making binary columns

In [23]:
def make_binary_columns(df_str, df_num, column_name):
    """
    Turns a single column named column_name (with various categories) into m binary columns, where m is the number
    of unique values in the original column. For each sample, the value for all new columns is 0 apart the one
    that matches the value of the original column. Names of new binary columns are formed as follows:
    column_name + '_' + str(original column value)
    Inputs:
        two pandas DataFrames: df_str where a single column contains information about given category
                               df_num that will later contain many binary columns
        column_name: name of the column that will be split into several binary columns
    Returns nothing. It appends the collumns directly into df_num to increase performance.
    """
    m = len(df_str)
    if (m != len(df_num)):
        raise Exception("df_str and df_num must have the same size.")
 
    categories = df_str[column_name].unique()

    from tqdm import tqdm_notebook # progress bar

    # Create a new feature for each category and initialize it to 0
    for i in tqdm_notebook(categories, desc='1/2'):
        df_num[column_name + '_' + str(i)] = np.zeros((m, 1), dtype=np.int8)

#     Loop thorugh all rows and assign 1 to the column whose name is the same as category
    for i in tqdm_notebook(df_str.index, desc='2/2'): # loop through all rows
        category = str(df_str.at[i, column_name])
        df_num.at[i, column_name + '_' + category] = 1

### brand_name: for each unique one create new binary feature

In [24]:
make_binary_columns(data, data_num, 'brand_name')

In [25]:
# data.head()

### main_cat, subcat_1, subcat_2: for each unique one create new binary feature

In [26]:
make_binary_columns(data, data_num, 'main_cat')
make_binary_columns(data, data_num, 'subcat_1')
make_binary_columns(data, data_num, 'subcat_2')

In [27]:
if (settings['del']):
    del data

# Split data and extract X, y and train_id

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(data_num, test_size = settings['test_size'], random_state=None) # randomly split data
# ! X_train and X_test STILL CONTAINS PRICE AT THIS MOMENT !

In [29]:
if (settings['del']):
    del data_num

In [30]:
# Pop price from X_train and X_test
y_train = X_train.pop('price')
y_test = X_test.pop('price')

In [31]:
# Pop id_train from both training and test data set

id_train = X_train.pop('train_id')
id_test = X_test.pop('train_id')

# Scale data

In [32]:
from sklearn.preprocessing import MaxAbsScaler

scaler = MaxAbsScaler()
scaler.fit(X_train) # Compute the mean adn std of training data to be used for future scaling

X_train_scaled = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
if (settings['del']):
    del X_train

X_test_scaled = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)
if (settings['del']):
    del X_test

In [33]:
# X_train_scaled.head()

In [34]:
# X_train.head()

In [35]:
# X_test_scaled.head()

In [36]:
# X_test.head()

In [37]:
who

MaxAbsScaler	 PATH	 X_test_scaled	 X_train_scaled	 id_test	 id_train	 item_description_len	 make_binary_columns	 np	 
pd	 scaler	 settings	 shuffle	 split_cat	 train_test_split	 y_test	 y_train	 


In [38]:
if (settings['save_env']):
    import dill                            #pip install dill --user
    dill.dump_session('splittedData.pkl')

# Check size of variables

In [39]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' By Fred Cirera, after https://stackoverflow.com/a/1094933/1870254'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name,value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name,sizeof_fmt(size)))

                X_train_scaled:   3.9GiB
                 X_test_scaled:   1.3GiB
          item_description_len:   8.1MiB
                       y_train:   2.3MiB
                      id_train:   2.3MiB
                        y_test: 781.3KiB
                       id_test: 781.3KiB
                             _:   3.8KiB
                           _15:   3.8KiB
                            _5:   2.1KiB


# Performance measure: RMSE

$$\text{RMSE} \left( \mathbf{Y} , \mathbf{\hat{Y}} \right) = \sqrt{\frac{1}{n} \sum_{i=1}^n \left( y_i - \hat{y_i} \right)^2 } $$

In [40]:
def rmse(y_test, y_pred):
    from sklearn.metrics import mean_squared_error
    return np.sqrt(mean_squared_error(y_test, y_pred))

# Apply linear regression

In [41]:
# Fit linear regression
from sklearn.linear_model import LinearRegression
lnr_regr = LinearRegression(n_jobs=-1)
print("Trainig...")
lnr_regr.fit(X_train_scaled, y_train)
print("Training done.")

if (settings['save_env']):
    dill.dump_session('linearModel.pkl')
    
# Make predictions and report train and test RMSEs

print("Evaluating performance on the training set...")
pred_train = lnr_regr.predict(X_train_scaled)
rmse_train = rmse(y_train, pred_train)
print("Training set RMSE: %.2f" % rmse_train)

print("Evaluating performance on the test test...")
pred_test = lnr_regr.predict(X_test_scaled)
rmse_test = rmse(y_test, pred_test)
print("Test set RMSE: %.2f" % rmse_test)

Trainig...
Training done.
Evaluating performance on the training set...
Training set RMSE: 31.32
Evaluating performance on the test test...
Test set RMSE: 411737971077.58


# Learning curves

In [ ]:
# import dill

In [ ]:
# dill.load_session('linearModel.pkl')

In [ ]:
# who